<a href="https://colab.research.google.com/github/fuyu-quant/langchain-examples/blob/main/ipynb/LangChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain
* 参考記事
    * https://note.com/npaka/n/n6b7a07e492f1

In [8]:
%%capture
!pip install langchain
!pip install openai
!pip install google-search-results
!pip install wikipedia
!pip install huggingface_hub

In [3]:
# パッケージのインポート
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI

import os

os.environ["LANGCHAIN_HANDLER"] = "langchain"


os.environ["OPENAI_API_KEY"] = ''
os.environ["SERPAPI_API_KEY"] = ""

In [21]:
#llm = OpenAI(temperature=0, model_name="text-davinci-002")
llm = OpenAI(temperature=0)

## ツール一覧
* python_relp
    * pythonコマンドを実行できる．
* serpapi
    * サーチエンジン，現在のイベントに関する質問に答える必要がある場合に役に立つ
* requests
    * サイトから特定のコンテンツを取得する場合に使う
    * 入力はURLで出力はそのページの全てのテキスト
* terminal
    * ターミナルでのコマンド実行
* llm-math
    * 数学の質問に答えるよう
* pal-math
    * 数学の質問に答えるよう
* pal-colored-objects
    * オブジェクトの位置と色についての推論に優れた言語モデル
* open-meteo-api
    * 気象情報を取得したい場合に使う
* news-api
    * 現在のニュース記事のトップ，ヘッドラインに関する情報を取得する
* tmdb-api
    * The Movie Databaseから情報を取得するのに使う
    

In [ ]:
tools = load_tools(["serpapi", "llm-math"], llm=llm)

## エージェント一覧
* https://langchain.readthedocs.io/en/latest/modules/agents/agents.html
* どのアクションをどのような順序で行うかを決めるもの
* zero-shot-react-description
    * 
* react-docstore
    * ドキュメントベースでの回答をする
* self-ask-with-search
    * 質問に対する事実に基づく回答を検索できる
* conversational-react-description
    * 会話用に最適化されたエージェント
    * 以前の会話のやり取りを記憶できる

In [4]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="zero-shot-react-description", 
    verbose=True
)

agent.run("興味を引くようなブレインパッドの広告を10個作ってください")

In [22]:
from langchain import Wikipedia
from langchain.agents import Tool
from langchain.agents.react.base import DocstoreExplorer

docstore=DocstoreExplorer(Wikipedia())
tools = [
    Tool(
        name="Search",
        func=docstore.search
    ),
    Tool(
        name="Lookup",
        func=docstore.lookup
    )
]


agent = initialize_agent(
    tools, 
    llm, 
    agent="react-docstore", 
    verbose=True
)

query = "株式会社ブレインパッドについて教えてください"

agent.run(query)

TypeError: ignored

In [11]:
agent = initialize_agent(
    tools, 
    llm, 
    agent="conversational-react-description", 
    verbose=True
)

input = {}

agent.run("興味を引くようなブレインパッドの広告を10個作ってください")

ValueError: ignored

### 似ているサンプルの作成

In [35]:
from langchain.llms.openai import OpenAI
from langchain.example_generator import generate_example
from langchain.prompts import PromptTemplate

In [ ]:
examples = [
    {
    "question": "株式会社ブレインパッドはいつ創業されいつ創業されましたか？",
     "answer": "a"
    },
    {
    "question": "株式会社ブレインパッドはいつ創業されいつ創業されましたか？",
     "answer": "a"
    },
     {
    "question": "株式会社ブレインパッドはいつ創業されいつ創業されましたか？",
     "answer": "a"
    }
]

In [ ]:
example_template = PromptTemplate(template="Question: {question}\n{answer}", input_variables=["question", "answer"])

new_example = generate_example(examples, OpenAI(), example_template)

new_example.split('\n')

## ユーティリティチェーン
* https://note.com/npaka/n/n886960b89de1

### LLMMathChain

In [77]:
from langchain import OpenAI, LLMMathChain

# LLMの準備
llm = OpenAI(temperature=0)

# LLMMathChainの準備
llm_math = LLMMathChain(
    llm=llm, 
    verbose=True
)

# LLMMathChainの実行
print(llm_math.run("How many of the integers between 72 and 354 inclusive are divisible by 8?"))



> Entering new LLMMathChain chain...
How many of the integers between 72 and 354 inclusive are divisible by 8?
Answer: 28
> Finished chain.
Answer: 28


### PALChain

In [73]:
from langchain.chains import PALChain
from langchain import OpenAI

# LLMの準備
llm = OpenAI(
    model_name='code-davinci-002', 
    temperature=0, 
    max_tokens=512)

# PALChainの準備
pal_chain = PALChain.from_math_prompt(
    llm, 
    verbose=True
)

# PALChainの実行
# (JanはMarciaの3倍のペットを飼っています。 MarciaはCindy より2匹多くペットを飼っています。シンディが4匹のペットを飼っている場合、3 人が飼っているペットの総数は?)
question = "Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"
print(pal_chain.run(question))



> Entering new PALChain chain...


def solution():
    """Jan has three times the number of pets as Marcia. Marcia has two more pets than Cindy. If Cindy has four pets, how many total pets do the three have?"""
    cindy_pets = 4
    marcia_pets = cindy_pets + 2
    jan_pets = marcia_pets * 3
    total_pets = cindy_pets + marcia_pets + jan_pets
    result = total_pets
    return result

> Finished chain.
28


### SQLDatabaseChain
* データベースに対するユーザーの質問を受け取り，SQLクエリを作成しその結果を取得してユーザーに返す．

In [74]:
from langchain import OpenAI, SQLDatabase, SQLDatabaseChain

# SQLDatabaseの準備
db = SQLDatabase.from_uri("sqlite:///../../../notebooks/Chinook.db")

# LLMの準備
llm = OpenAI(temperature=0)

# SQLDatabaseChainの準備
db_chain = SQLDatabaseChain(llm=llm, database=db, verbose=True)

# SQLDatabaseChainの実行
# (従業員は何人ですか？)
print(db_chain.run("How many employees are there?"))

OperationalError: ignored

### LLMBashChain

### LLMCheckerChain

### LLMRequestsChain

### OpenAIModerationChain